In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm
import pandas as pd
from data_sources import process_parallel, transform_csv_row

In [4]:
from collections import Counter

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [5]:
version = 'v4'
data_path = Path('data/midi')
version_path = data_path/version

In [8]:
source_dir = 'midi_npz'

out_dir = 'midi_transcribe/longcont'
continuous = True
short=False
def note_enc(note):
    if continuous: return note.continuous_repr(short=short)
    return note.duration_repr(short=short)

source_csv = version_path/source_dir/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'midi_transcribe.csv'

In [9]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,4,9,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,genres,ht_offset,source,song_url,parts,seconds,inferred_offset,title,time_signature,ht_key,...,midi_title,ht_bpm,bpm,midi,inferred_key,quarter_length,mxl,midi_transform,midi_npz,midi_npz_timesteps
0,NaN,0.0,hooktheory,https://www.hooktheory.com/theorytab/view/wayn...,"intro,chorus",25.411765,0.0,yu-gi-oh-theme-song,3/4,C,...,yu-gi-oh,85.0,85.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36.0,NaN,NaN,NaN,NaN
1,Holiday,-1.0,hooktheory,https://www.hooktheory.com/theorytab/view/wham...,"intro,verse,chorus",17.777778,-1.0,last-christmas,4/4,Db,...,Last Christmas Chorus,108.0,108.0,midi_sources/hooktheory/pianoroll/w/wham/last-...,B- minor,32.0,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...,midi_npz/hooktheory/pianoroll/w/wham/last-chri...,NaN
2,NaN,6.0,hooktheory,https://www.hooktheory.com/theorytab/view/wolf...,verse,15.118110,1.0,love-and-war,4/4,G#,...,wolfgang gartner love and war,127.0,127.0,midi_sources/hooktheory/pianoroll/w/wolfgang-g...,G# minor,32.0,NaN,midi_transform/hooktheory/pianoroll/w/wolfgang...,midi_npz/hooktheory/pianoroll/w/wolfgang-gartn...,NaN
3,Electronic,3.0,hooktheory,https://www.hooktheory.com/theorytab/view/wood...,"intro,bridge",12.972973,-2.0,rainbow-factory,4/4,F#,...,Rainbow Factory,148.0,148.0,midi_sources/hooktheory/pianoroll/w/woodentoas...,D major,32.0,NaN,midi_transform/hooktheory/pianoroll/w/woodento...,midi_npz/hooktheory/pianoroll/w/woodentoaster/...,NaN
4,NaN,8.0,hooktheory,https://www.hooktheory.com/theorytab/view/wolf...,pre-chorus,15.000000,1.0,space-junk,4/4,F#,...,Space Junk,128.0,128.0,midi_sources/hooktheory/pianoroll/w/wolfgang-g...,B major,32.0,NaN,midi_transform/hooktheory/pianoroll/w/wolfgang...,midi_npz/hooktheory/pianoroll/w/wolfgang-gartn...,NaN


In [10]:
df_filtered = df.loc[df[source_dir].notna()]; df_filtered.shape

(31621, 25)

In [11]:
def transform_func(file, out_file, row):
    chordarr = load_chordarr(file)
    seq = chordarr2seq(chordarr)
    string_repr = seq2str(seq, note_enc, continuous=continuous)
    with open(out_file, 'w') as tf:
        tf.write(string_repr)

In [12]:
from functools import partial
parallel_func = partial(transform_csv_row, 
        transform_func=transform_func,
        base_path=version_path,
        source_dir=source_dir,
        out_dir=out_dir,
        out_extension='.txt'
       )

In [ ]:
# def transform_row(idxrow, transform_func, base_path, source_dir, out_dir, extension):
#     idx,row = idxrow
#     file = row[source_dir]
    
#     if not isinstance(file, str): return idx,None
#     file = Path(base_path)/file
#     if not file.exists(): return idx, None
    
#     out_file = Path(str(file).replace(f'/{source_dir}/', f'/{out_dir}/')).with_suffix(extension)
#     out_file.parent.mkdir(parents=True, exist_ok=True)
#     if out_file.exists(): return idx,str(out_file.relative_to(base_path))
#     try: transform_func(file, out_file, row)
#     except Exception as e:
#         print('Error converting midi to sequence', e)
#         return idx,None
#     return idx,str(out_file.relative_to(base_path))

In [9]:
def transcribe_file(idxrow, base_path):
    idx,row = idxrow
    file = row[source_dir]
    
    if not isinstance(file, str): return idx,None
    file = Path(base_path)/file
    if not file.exists(): return idx, None
    
    if row[f'{source_dir}_timesteps'] > 2e4: return idx,None
    out_file = Path(str(file).replace(f'/{source_dir}/', f'/{out_dir}/')).with_suffix('.txt')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return idx,str(out_file.relative_to(base_path))
    try:
        chordarr = load_chordarr(file)
        seq = chordarr2seq(chordarr)
        string_repr = seq2str(seq, note_enc, continuous=continuous)
        with open(out_file, 'w') as tf:
            tf.write(string_repr)
    except Exception as e:
        print('Error converting midi to sequence', e)
        return idx,None
    return idx,str(out_file.relative_to(base_path))

In [14]:
for r in df_filtered.iterrows():
    parallel_func(r)
    break

NameError: name 'extension' is not defined

In [13]:
transcribed_files = process_parallel(parallel_func, df_filtered.iterrows(), total=df_filtered.shape[0])

Process ForkProcess-43:
Process ForkProcess-37:
Process ForkProcess-31:
Process ForkProcess-77:
Process ForkProcess-40:
Process ForkProcess-38:
Process ForkProcess-25:
Process ForkProcess-95:
Process ForkProcess-44:
Process ForkProcess-53:
Process ForkProcess-27:
Process ForkProcess-39:
Process ForkProcess-46:
Process ForkProcess-87:
Process ForkProcess-58:
Process ForkProcess-30:
Process ForkProcess-47:
Process ForkProcess-79:
Process ForkProcess-65:
Process ForkProcess-45:
Process ForkProcess-18:
Process ForkProcess-13:
Process ForkProcess-48:
Process ForkProcess-62:
Process ForkProcess-20:
Process ForkProcess-94:
Process ForkProcess-28:
Process ForkProcess-42:
Process ForkProcess-84:
Process ForkProcess-29:
Process ForkProcess-85:
Process ForkProcess-78:
Process ForkProcess-68:
Process ForkProcess-26:


KeyboardInterrupt: 

Process ForkProcess-76:
Process ForkProcess-63:
Process ForkProcess-89:
Process ForkProcess-17:
Process ForkProcess-55:
Process ForkProcess-92:
Process ForkProcess-32:
Process ForkProcess-83:
Process ForkProcess-64:
Process ForkProcess-36:
Process ForkProcess-11:
Process ForkProcess-15:
Process ForkProcess-93:
Process ForkProcess-50:
Process ForkProcess-61:
Process ForkProcess-86:
Process ForkProcess-10:
Process ForkProcess-54:
Process ForkProcess-41:
Process ForkProcess-82:
Process ForkProcess-23:
Process ForkProcess-49:
Process ForkProcess-19:
Process ForkProcess-52:
Process ForkProcess-34:
Process ForkProcess-69:
Process ForkProcess-56:
Process ForkProcess-51:
Process ForkProcess-2:
Process ForkProcess-81:
Process ForkProcess-24:
Process ForkProcess-22:
Process ForkProcess-66:
Process ForkProcess-4:
Process ForkProcess-3:
Process ForkProcess-75:
Process ForkProcess-12:
Process ForkProcess-5:
Process ForkProcess-1:
Process ForkProcess-21:
Process ForkProcess-59:
Process ForkProcess-1

Process ForkProcess-73:
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    sel

  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._ta

...

  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ubuntu/anaconda3/envs/midi/lib/p

  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/envs/mid

  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/ubuntu/anaconda3/e

  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    retu

  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__ent

In [12]:
tdf = pd.DataFrame(data={out_dir: list(transcribed_files.values())}, index=list(transcribed_files.keys()))

In [13]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

((33262, 1), (39929, 25), (39929, 26))

In [14]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

,source,artist,title,ht_key,parts,ht_offset,ht_bpm,midi,genres,time_signature,...,ht_mode,instruments,bpm,ht_time_signature,inferred_key,mxl,midi_transform_v3,midi_npz_v3,midi_npz_v3_timesteps,midi_transcribe_v3_longcont
0,hooktheory,weezer,beverly-hills,C,intro-and-verse,0.0,128.0,data/midi/midi_sources_v3/hooktheory/pianoroll...,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1,hooktheory,willie-nelson,on-the-road-again,E,"verse,bridge",-4.0,112.0,data/midi/midi_sources_v3/hooktheory/pianoroll...,NaN,4/4,...,1.0,Piano,112.0,4.0,E major,NaN,data/midi/midi_transform_v3/hooktheory/pianoro...,data/midi/midi_npz_v3/hooktheory/pianoroll/w/w...,113.0,data/midi/midi_transcribe_v3_longcont/hooktheo...
2,hooktheory,weebl,donkeys,B,"intro,verse",1.0,140.0,data/midi/midi_sources_v3/hooktheory/pianoroll...,NaN,4/4,...,1.0,Piano,140.0,4.0,F# major,NaN,data/midi/midi_transform_v3/hooktheory/pianoro...,data/midi/midi_npz_v3/hooktheory/pianoroll/w/w...,129.0,data/midi/midi_transcribe_v3_longcont/hooktheo...
3,hooktheory,wayne-sharpe,yu-gi-oh-theme-song,C,"intro,chorus",0.0,128.0,data/midi/midi_sources_v3/hooktheory/pianoroll...,NaN,4/4,...,1.0,"Piano,Piano",128.0,4.0,C major,NaN,data/midi/midi_transform_v3/hooktheory/pianoro...,data/midi/midi_npz_v3/hooktheory/pianoroll/w/w...,129.0,data/midi/midi_transcribe_v3_longcont/hooktheo...
4,hooktheory,wham,last-christmas,Db,"intro,verse,chorus",-1.0,108.0,data/midi/midi_sources_v3/hooktheory/pianoroll...,Holiday,4/4,...,1.0,"Piano,Piano",108.0,4.0,B- minor,NaN,data/midi/midi_transform_v3/hooktheory/pianoro...,data/midi/midi_npz_v3/hooktheory/pianoroll/w/w...,129.0,data/midi/midi_transcribe_v3_longcont/hooktheo...
